Reference - https://medium.com/@jainashish.079/build-llm-agent-combining-reasoning-and-action-react-framework-using-langchain-379a89a7e881

Most applications will require the LLM not to do only reasoning, even reasoning steps might requires calling multiple external data sources and applications. One of the techniques or framework is called ReAct (Reasoning and actions) in which LLM can plan out these reasoning steps and execute those steps.

ReAct is technique which enable LLMs to do reasoning and take task specific actions. It combines chain of thought reasoning with action planning.
ReAct agents can perform 3 major actions : search about entities, look up strings, and finish current task with an answer once found

Langchain's DocStoreExplorer agent interacts with a document storage system (like Wikipedia), using two specific tools , a Search tool and a Lookup tool. The Search tool is responsible for locating a document, whereas the Lookup tool retrieves a term from the most recently discovered document. We can initialize this doc store with Wikipedia as a document store.

In [ ]:
import openai
import os

from langchain.agents.react.base import DocstoreExplorer
from langchain.docstore import Wikipedia
from langchain.agents import Tool
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain_community.llms import HuggingFaceEndpoint
from keys import HUGGING_FACE_KEY

# If loading from OpenAI using env
os.getenv('OPENAI_API_KEY')

# If loading from Hugging Face using env
os.getenv('HUGGING_FACE_KEY')

# 1. Using AutoTokenizer, AutoModelForCausalLM Classes

## Downloads LLM + Tokenizer locally

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

#If using HuggingFace model for the first time
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", cache_dir="D:/HuggingFaceTokenizerCache/")
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", cache_dir="D:/HuggingFaceModelCache/")

tokenizer = AutoTokenizer.from_pretrained("D:/HuggingFaceTokenizerCache/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873")
model = AutoModelForCausalLM.from_pretrained("D:/HuggingFaceModelCache/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873", low_cpu_mem_usage=True)

model_inputs = tokenizer(["Who is the current president of Germany?"], return_tensors="pt") #tokenize the string
generated_ids = model.generate(**model_inputs, max_new_tokens=100) #use model to generate 100 new tokens based on the tokenized input
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0] #use the same tokenizer to decode the generated output from model

# 2. Using HuggingFaceEndpoints

## No local downloads of LLMs

In [ ]:
llm = HuggingFaceEndpoint(
    huggingfacehub_api_token=HUGGING_FACE_KEY,
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    task="text-generation",
    max_new_tokens=512,
    top_k=50,
    top_p=0.8,
    temperature=0.1,
    repetition_penalty=1
)

## Setting up reAct tools needed for agent

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import PromptTemplate

In [ ]:
tools = load_tools(["ddg-search"])
prompt = hub.pull("hwchase17/react")

## Create and run reAct agent

In [ ]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "What does Dale Steyn and Kumar Sangakkara have in common?", "use_gpu": True})